In [1]:
import requests
from bs4 import BeautifulSoup
import csv


global stats
stats = []
global stop_factor
global scored_tables
global missed_tables
global name

In [2]:
# function retrieves links to all tables of data for a soccer player on transfermarkt
def get_links(page):
    global stop_factor
    print('---start getting links---')
    headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    response = requests.get(page, headers=headers)
    content = BeautifulSoup(response.content, 'html.parser')
    list_of_links = content.find_all('ul', {'class': 'tm-pagination'})
    next_page_links = []
    pen_scored_links = list_of_links[0].find_all('a', {'class': 'tm-pagination__link'})
    pen_missed_links = list_of_links[1].find_all('a', {'class': 'tm-pagination__link'})
    stop_factor = len(pen_missed_links)-2 # number of tables for pen_missed is smaller than pen_scored
    webpage = 'https://www.transfermarkt.us'
    for i in range(0, len(pen_scored_links)-2): # ignore next page & last page links
        next_page_links.append(webpage + pen_scored_links[i]['href'])
    return next_page_links

In [3]:
list_of_links = get_links('https://www.transfermarkt.us/cristiano-ronaldo/elfmetertore/spieler/8198/saison_id//wettbewerb_id//plus/1#verschossen')
list_of_links

---start getting links---


['https://www.transfermarkt.us/cristiano-ronaldo/elfmetertore/spieler/8198/saison_id//wettbewerb_id//plus/1',
 'https://www.transfermarkt.us/cristiano-ronaldo/elfmetertore/spieler/8198/saison_id//wettbewerb_id//plus/1/page/2',
 'https://www.transfermarkt.us/cristiano-ronaldo/elfmetertore/spieler/8198/saison_id//wettbewerb_id//plus/1/page/3',
 'https://www.transfermarkt.us/cristiano-ronaldo/elfmetertore/spieler/8198/saison_id//wettbewerb_id//plus/1/page/4',
 'https://www.transfermarkt.us/cristiano-ronaldo/elfmetertore/spieler/8198/saison_id//wettbewerb_id//plus/1/page/5',
 'https://www.transfermarkt.us/cristiano-ronaldo/elfmetertore/spieler/8198/saison_id//wettbewerb_id//plus/1/page/6',
 'https://www.transfermarkt.us/cristiano-ronaldo/elfmetertore/spieler/8198/saison_id//wettbewerb_id//plus/1/page/7',
 'https://www.transfermarkt.us/cristiano-ronaldo/elfmetertore/spieler/8198/saison_id//wettbewerb_id//plus/1/page/8',
 'https://www.transfermarkt.us/cristiano-ronaldo/elfmetertore/spieler/8

In [4]:
def get_tables(links):
    global scored_tables
    scored_tables = []
    global missed_tables
    missed_tables = []
    global stop_factor
    global name
    i = 0
    name = links[0].split("/")[3] # get the player's name
    headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    for l in links:
        response = requests.get(l, headers=headers)
        content = BeautifulSoup(response.content, 'html.parser')
        scored_tables.append(content.findAll('table')[1].findAll('tr'))
        if i < stop_factor:
            missed_tables.append(content.findAll('table')[3].findAll('tr'))
            i = i + 1

In [5]:
get_tables(list_of_links)

In [6]:
def get_stats(tables, missed):
    global stats
    string = []
    for table in tables: 
        for rows in table:     
            cols = rows.find_all('td')
            if len(cols) != 0:
                if missed == True:
                    string.append(0)
                else:
                    string.append(1)
                i = 0
                for col in cols:
                    if len(col.text) > 0:
                        string.append(col.text.strip())
                        if i == 9:
                            string.append('https://www.transfermarkt.us' + col.find('a').get('href'))
                    else: 
                        string.append(col.find('a').get('title'))
                    i = i + 1
                stats.append(string)
                string = []

In [7]:
get_stats(scored_tables, False)
get_stats(missed_tables, True)

In [21]:
# writing to csv file
with open(name + '-data.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['pen-scored','season', 'competition', 'club', 'date', 'home-team','final-result','away-team', 'minute','score-before-pk','goal-keeper','goal-keeper-link'])
        writer.writerows(stats)
print(name + '-data.csv done!')

cristiano-ronaldo-data.csv done!


## Calculate statistical information based on the player's data

In [12]:
from datascience import *
import numpy as np
data = Table.read_table("cristiano-ronaldo-data.csv")
data

pen-scored,season,competition,club,date,home-team,final-result,away-team,minute,score-before-pk,goal-keeper,goal-keeper-link
1,22/23,Saudi Pro League,Al-Nassr FC,"Apr 4, 2023",Al-Adalah FC,0:5,Al-Nassr FC,40',0:1,Milan Mijatovic,https://www.transfermarkt.us/milan-mijatovic/elfmetersta ...
1,22/23,European Qualifiers,Portugal,"Mar 23, 2023",Portugal,4:0,Liechtenstein,51',3:0,Benjamin Büchel,https://www.transfermarkt.us/benjamin-buchel/elfmetersta ...
1,22/23,Saudi Pro League,Al-Nassr FC,"Feb 25, 2023",Damac FC,0:3,Al-Nassr FC,18',0:1,Moustapha Zeghba,https://www.transfermarkt.us/moustapha-zeghba/elfmeterst ...
1,22/23,Saudi Pro League,Al-Nassr FC,"Feb 9, 2023",Al-Wehda FC,0:4,Al-Nassr FC,53',0:3,Abdulqaddous Attieh,https://www.transfermarkt.us/abdulqaddous-attieh/elfmete ...
1,22/23,Saudi Pro League,Al-Nassr FC,"Feb 3, 2023",Al-Fateh SC,2:2,Al-Nassr FC,90',2:2,Jacob Rinne,https://www.transfermarkt.us/jacob-rinne/elfmeterstatist ...
1,21/22,2022 World Cup,Portugal,"Nov 24, 2022",Portugal,3:2,Ghana,65',1:0,Lawrence Ati Zigi,https://www.transfermarkt.us/lawrence-ati-zigi/elfmeters ...
1,22/23,Europa League,Manchester United,"Sep 15, 2022",FC Sheriff Tiraspol,0:2,Manchester United,39',0:2,Maksym Koval,https://www.transfermarkt.us/maksym-koval/elfmeterstatis ...
1,21/22,Premier League,Manchester United,"May 2, 2022",Manchester United,3:0,Brentford FC,61',2:0,David Raya,https://www.transfermarkt.us/david-raya/elfmeterstatisti ...
1,21/22,Premier League,Manchester United,"Dec 11, 2021",Norwich City,0:1,Manchester United,75',0:1,Tim Krul,https://www.transfermarkt.us/tim-krul/elfmeterstatistik/ ...
1,21/22,Premier League,Manchester United,"Dec 2, 2021",Manchester United,3:2,Arsenal FC,70',3:2,Aaron Ramsdale,https://www.transfermarkt.us/aaron-ramsdale/elfmeterstat ...


In [14]:
# calculate the goalkeeper's historical save rate
def saved_penalty_ratio(page):
    headers = {'User-Agent': 
               'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    response = requests.get(page, headers=headers)
    content = BeautifulSoup(response.content, 'html.parser')
    h2 = content.find_all('h2', {'class': 'content-box-headline'})
    saved_penalty = int(h2[0].text.strip().split('-')[1])
    non_saved_penalty = int(h2[1].text.strip().split('-')[2])
    return saved_penalty/(saved_penalty + non_saved_penalty)
ratio = data.apply(saved_penalty_ratio, "goal-keeper-link")

In [19]:
pen_scored = data.column('pen-scored')
rows = []
for i in range(0,data.num_rows):
    col = [0] * 3
    col[0] = pen_scored[i]
    score = data.column('score-before-pk')[i].split(":")
    # calculate the goal difference before the penalty kick
    if data.column('club')[i] == data.column('home-team')[i]:
        col[1] = int(score[0]) - int(score[1])
    else:
        col[1] = int(score[1]) - int(score[0])
    col[2] = ratio[i]
    rows.append(col)

In [20]:
with open(name + '-stats.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['pen-scored','score-diff','goal-keeper-success-rate'])
        writer.writerows(rows)
print(name + '-stats.csv done!')

cristiano-ronaldo-stats.csv done!
